<a href="https://colab.research.google.com/github/ghommidhWassim/hate_speech_detection/blob/main/hate_speech_detection.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [24]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import accuracy_score
from sklearn import tree
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.metrics import accuracy_score, classification_report, log_loss


In [5]:
import re
import nltk
from nltk.util import pr
from nltk.corpus import stopwords
import string
nltk.download('stopwords')
stemmer = nltk.SnowballStemmer("english")
stopword=set(stopwords.words('english'))


[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


In [12]:
def clean_text(text):
    text = text.lower()
    text = re.sub(r'\[.*?\]', '', text)
    text = re.sub(r'https?://\S+|www\.\S+', '', text)
    text = re.sub(r'<.*?>+', '', text)
    text = re.sub(r'[%s]' % re.escape(string.punctuation), '', text)
    text = re.sub(r'\n', '', text)
    text = re.sub(r'\w*\d\w*', '', text)
    words = [stemmer.stem(word) for word in text.split() if word not in stopword]
    return " ".join(words)

In [7]:
df = pd.read_csv("twitter_data.csv")
df

,Unnamed: 0,count,hate_speech,offensive_language,neither,class,tweet
0,0,3,0,0,3,2,!!! RT @mayasolovely: As a woman you shouldn't...
1,1,3,0,3,0,1,!!!!! RT @mleew17: boy dats cold...tyga dwn ba...
2,2,3,0,3,0,1,!!!!!!! RT @UrKindOfBrand Dawg!!!! RT @80sbaby...
3,3,3,0,2,1,1,!!!!!!!!! RT @C_G_Anderson: @viva_based she lo...
4,4,6,0,6,0,1,!!!!!!!!!!!!! RT @ShenikaRoberts: The shit you...
...,...,...,...,...,...,...,...
24778,25291,3,0,2,1,1,you's a muthaf***in lie &#8220;@LifeAsKing: @2...
24779,25292,3,0,1,2,2,"you've gone and broke the wrong heart baby, an..."
24780,25294,3,0,3,0,1,young buck wanna eat!!.. dat nigguh like I ain...
24781,25295,6,0,6,0,1,youu got wild bitches tellin you lies


In [9]:
df["labels"]= df["class"].map({0: "Hate Speech detected", 1: "Offensive Language detected", 2: "No Hate and Offensive"})
df

,Unnamed: 0,count,hate_speech,offensive_language,neither,class,tweet,labels
0,0,3,0,0,3,2,!!! RT @mayasolovely: As a woman you shouldn't...,No Hate and Offensive
1,1,3,0,3,0,1,!!!!! RT @mleew17: boy dats cold...tyga dwn ba...,Offensive Language detected
2,2,3,0,3,0,1,!!!!!!! RT @UrKindOfBrand Dawg!!!! RT @80sbaby...,Offensive Language detected
3,3,3,0,2,1,1,!!!!!!!!! RT @C_G_Anderson: @viva_based she lo...,Offensive Language detected
4,4,6,0,6,0,1,!!!!!!!!!!!!! RT @ShenikaRoberts: The shit you...,Offensive Language detected
...,...,...,...,...,...,...,...,...
24778,25291,3,0,2,1,1,you's a muthaf***in lie &#8220;@LifeAsKing: @2...,Offensive Language detected
24779,25292,3,0,1,2,2,"you've gone and broke the wrong heart baby, an...",No Hate and Offensive
24780,25294,3,0,3,0,1,young buck wanna eat!!.. dat nigguh like I ain...,Offensive Language detected
24781,25295,6,0,6,0,1,youu got wild bitches tellin you lies,Offensive Language detected


In [13]:
df=df[['tweet','labels']]
df['tweet']=df['tweet'].apply(clean_text)
df

,tweet,labels
0,rt mayasolov woman shouldnt complain clean hou...,No Hate and Offensive
1,rt boy dat coldtyga dwn bad cuffin dat hoe place,Offensive Language detected
2,rt urkindofbrand dawg rt ever fuck bitch start...,Offensive Language detected
3,rt cganderson vivabas look like tranni,Offensive Language detected
4,rt shenikarobert shit hear might true might fa...,Offensive Language detected
...,...,...
24778,yous muthafin lie coreyemanuel right tl trash ...,Offensive Language detected
24779,youv gone broke wrong heart babi drove redneck...,No Hate and Offensive
24780,young buck wanna eat dat nigguh like aint fuck...,Offensive Language detected
24781,youu got wild bitch tellin lie,Offensive Language detected


In [20]:
x= np.array(df['tweet'])
y= np.array(df['labels'])
cv = CountVectorizer()
x=cv.fit_transform(x)

In [21]:
X_train, X_test, Y_train, Y_test= train_test_split(x,y, test_size=0.33, random_state=42)
dtc= DecisionTreeClassifier()
dtc.fit(X_train, Y_train)


DecisionTreeClassifier()

In [23]:
y_pred= dtc.predict(X_test)
y_pred

array(['Offensive Language detected', 'Offensive Language detected',
       'Offensive Language detected', ..., 'Offensive Language detected',
       'Offensive Language detected', 'Offensive Language detected'],
      dtype=object)

In [26]:
acc = accuracy_score(Y_test, y_pred)
print("Accuracy :", acc)

#Rapport détaillé (Precision, Recall, F1-score)
print("\nClassification Report:\n", classification_report(Y_test, y_pred))

#Log Loss
y_pred_proba = dtc.predict_proba(X_test)
loss = log_loss(Y_test, y_pred_proba)
print("\nLog Loss :", loss)

# error
error_rate = 1 - acc
print("\nError rate :", error_rate)


Accuracy : 0.87504584912581

Classification Report:
                              precision    recall  f1-score   support

       Hate Speech detected       0.38      0.35      0.36       465
      No Hate and Offensive       0.80      0.83      0.81      1379
Offensive Language detected       0.93      0.92      0.93      6335

                   accuracy                           0.88      8179
                  macro avg       0.70      0.70      0.70      8179
               weighted avg       0.87      0.88      0.87      8179


Log Loss : 4.420387646672523

Error rate : 0.12495415087419004


In [35]:
test_data ="I don't like you, i hate you"
df = cv.transform([test_data]).toarray()
print(dtc.predict(df))

['Offensive Language detected']
